In [16]:
import re
from collections import Counter

In [17]:
name = "Top inventory management apps for Shopify [2243]"
text = open("C:/Users/TonyM/Downloads/"+name+".txt", encoding="utf8").read()

keywords = """inventory management apps
shopify inventory management app
inventory management shopify
best shopify inventory management
best inventory management software shopify
shopify inventory management software
shopify inventory management apps
inventory management for shopify
shopify inventory app
best inventory apps
apps for inventory
best inventory management for shopify
inventory management software for shopify
stock management app for shopify
shopify inventory apps
store inventory app
best app for inventory
inventory app for shopify"""

In [18]:
def keywordcheckfunc(name, text, keywords):
    all_occurances_dic = {}
    keywords_list = keywords.split('\n')
    def lenfunc(e):
        return len(e)
    keywords_list.sort(reverse=True, key=lenfunc) 
    #move longer words to the top of the list to be counted first 
    #otherwise nested keywords (validate product --> validate product idea --> how to validate product idea) may be counted incorrectly
    for keyword in keywords_list:
        all_occurances_dic[keyword]=len(re.findall(keyword, text, flags=re.IGNORECASE)) #count all occurances as in the v1
    cross_section_dic = {} #dict of lists for keywords (keyword1) included into other keywords (keyword2)
    for keyword1 in keywords_list:
        sub_dic = []
        for keyword2 in keywords_list:
            if keyword1!=keyword2 and keyword1 in keyword2:
                sub_dic.append(keyword2)
        if len(sub_dic)>0:
            cross_section_dic[keyword1] = sub_dic
    #subtract occurances of long keywords from occurances of short ones if the former include the latter
    for keyword1 in cross_section_dic.keys():
        counter = 0
        for keyword2 in cross_section_dic[keyword1]:
            counter +=all_occurances_dic[keyword2]
        all_occurances_dic[keyword1] = all_occurances_dic[keyword1] - counter
    return all_occurances_dic

In [22]:
keywordcheck_dict = keywordcheckfunc(name, text, keywords)

In [23]:
keywordcheck_dict

{'best inventory management software shopify': 0,
 'inventory management software for shopify': 4,
 'shopify inventory management software': 2,
 'best inventory management for shopify': 1,
 'best shopify inventory management': 3,
 'shopify inventory management apps': 4,
 'shopify inventory management app': 2,
 'inventory management for shopify': 2,
 'stock management app for shopify': 1,
 'inventory management shopify': 1,
 'inventory management apps': 3,
 'inventory app for shopify': 2,
 'shopify inventory apps': 2,
 'best app for inventory': 2,
 'shopify inventory app': 2,
 'best inventory apps': 3,
 'store inventory app': 2,
 'apps for inventory': 3}

# Partial matches

Sometimes keywords are slightly changed (e.g., for readability, grammar correctness) and can't be matched directly. For examples, "UI UX design problems" - "UI/UX design problems". This section will help to deal with such cases

In [24]:
targets_for_partial_matches_search = []

In [25]:
for keyword, occurances in keywordcheck_dict.items():
    if occurances == 0:
        targets_for_partial_matches_search.append(keyword)

In [26]:
from difflib import SequenceMatcher

In [27]:
wordlist = text.split()

In [28]:
def find_partial_matches (target):
    target_len = len(target.split())
    length_diff_list = [-2, 0 , 2] #the n-gram we are looking for can be larger or smaller than the target, 
    #e.g. increase sales shopify (3 words) --> increase sales ON shopify (4 words) are similar, but have different lengths
    text_to_compare_list = []
    similar_text_dict = {}
    for length_diff in length_diff_list:
        for n in range(len(wordlist)-(target_len+length_diff)):
            text_to_compare = " ".join(wordlist[n:n+(target_len+length_diff)]) #n-gram to check against target n-gram
            if text_to_compare not in text_to_compare_list:
                text_to_compare_list.append(text_to_compare)
                seq_match = SequenceMatcher(None, target, text_to_compare)
                ratio = seq_match.ratio()
#                 if ratio>0.5: #checking ratios againsts arbitrary threshhold
                similar_text_dict[text_to_compare] = ratio
#     similar_text_dict_sorted = dict(Counter(similar_text_dict).most_common()) #sorting the dictionary by values
    closest_match = max(similar_text_dict, key=similar_text_dict.get)
    return(closest_match)

In [29]:
for target in targets_for_partial_matches_search:
    closest_match = find_partial_matches (target)
    closest_match_occurances=len(re.findall(closest_match, text, flags=re.IGNORECASE))
    keywordcheck_dict[target] = "0, closest_match = " + closest_match + " ("+ str(closest_match_occurances) + " occurances)"

In [30]:
keywordcheck_dict

{'best inventory management software shopify': '0, closest_match = best inventory management software for Shopify x (1 occurances)',
 'inventory management software for shopify': 4,
 'shopify inventory management software': 2,
 'best inventory management for shopify': 1,
 'best shopify inventory management': 3,
 'shopify inventory management apps': 4,
 'shopify inventory management app': 2,
 'inventory management for shopify': 2,
 'stock management app for shopify': 1,
 'inventory management shopify': 1,
 'inventory management apps': 3,
 'inventory app for shopify': 2,
 'shopify inventory apps': 2,
 'best app for inventory': 2,
 'shopify inventory app': 2,
 'best inventory apps': 3,
 'store inventory app': 2,
 'apps for inventory': 3}